In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
import pandas as pd
import shutil

Combining a real dataset with a synthetic dataset

In [3]:
synthetic_dir = repo_path / 'generation/inpainting/data/split_1' / f'improved_normal' #<----- change

original_training_metadata = pd.read_csv(repo_path / 'data/CDD-CESM/metadata/bboxes/split_1/train_set.csv')
synt_trianing_metadata = pd.read_csv(synthetic_dir / 'metadata.csv')
print(f'The orignal length of the training set is {len(original_training_metadata)}')
print(f'The length of the synthetic training set is {len(synt_trianing_metadata)}')

combined_metadata = pd.concat([original_training_metadata, synt_trianing_metadata], ignore_index=True)
combined_metadata.sort_values(by='patient_id', inplace=True)

# create copy of all images in a new dir
real_imdir = repo_path / 'data/CDD-CESM/images/substracted'
synthetic_imdir = synthetic_dir / 'images'
new_dir = repo_path / 'detection/preprocessing/data/combined_datasets/split_1/improved_augmentation' #<----- change
(new_dir / 'images').mkdir(parents=True, exist_ok=True)

combined_metadata.to_csv(new_dir / 'metadata.csv', index=False)
for i, row in original_training_metadata.iterrows():
    original_path = real_imdir / (row['image_name'].replace(' ', '') + '.jpg') # remove spaces from the filename
    shutil.copy(original_path, new_dir / 'images' / original_path.name)
for i, row in synt_trianing_metadata.iterrows():
    original_path = synthetic_imdir / (row['image_name'].replace(' ', '') + '.jpg') # remove spaces from the filename
    shutil.copy(original_path, new_dir / 'images' / original_path.name)
    

The orignal length of the training set is 312
The length of the synthetic training set is 415
